In [1]:
import numpy as np 
# import torch 

In [2]:
def read_txt(path):
    with open(path, 'r') as f:
        train_data = f.readlines()
        train_data = [x.strip() for x in train_data]
        train_data = [[int(j) for j in i.split()] for i in train_data]
        print(f'train sample: {len(train_data)}') # num user 
    return train_data
  
train_data = read_txt('train.txt')
test_data = read_txt('test.txt')
val_data = read_txt('val.txt')

train sample: 19442
train sample: 10474
train sample: 10342


In [3]:
# text_f = np.load('./text_feat.npy', allow_pickle=True)
# img_f = np.load('./image_feat.npy', allow_pickle=True)
# print(f'txt feat: {text_f.shape} \nimg feat: {img_f.shape}')

# text_f_missing_all = np.load('./text_feat_missing_all.npy', allow_pickle=True)
# print(f'text missing all: {text_f_missing_all.shape}')

In [4]:
# text_f_missing_test = np.load('./text_feat_missing_test.npy', allow_pickle=True)
# print(f'text missing test: {text_f_missing_test.shape}')
# print(text_f_missing_test.sum(axis=1).nonzero()[0].shape)
# print(f'sample: {text_f_missing_test[:5]}')
# print('-'*50)
# text_f_missing_train = np.load('./text_feat_missing_train.npy', allow_pickle=True)
# print(f'text missing train: {text_f_missing_train.shape}')
# print(text_f_missing_train.sum(axis=1).nonzero()[0].shape)
# print(f'sample: {text_f_missing_train[:5]}')

In [5]:
# from numpy.linalg import norm

# def cosine_sim(x, y):
#     return np.dot(x, y)/ (norm(x) * norm(y))

# image_feat_missing_test = np.load('./image_feat_missing_test.npy', allow_pickle=True)
# image_feat_missing_train = np.load('./image_feat_missing_train.npy', allow_pickle=True)

# print(f'image feat missing test full:\n {image_feat_missing_test}')
# print(f'partial image feat missing test:\n {image_feat_missing_test[0][:10]}')

# print(f'image feat missing train full:\n {image_feat_missing_train}')
# print(f'partial image feat missing train:\n {image_feat_missing_train[0][:10]}')

# print('-'*50)

# # print(f'sim: {cosine_sim(
# #     image_feat_missing_test[0], image_feat_missing_train[0]
# # )}')


In [6]:
warm_missing_i_idx = np.load('./warm_missing_item_index.npy', allow_pickle=True)
print(f'warm missing item: {warm_missing_i_idx.shape}')
print(warm_missing_i_idx[:10])



warm missing item: (3947,)
[3554 1948  223 6880 2194 6747 5700 5913 4348  697]


In [7]:
# cold item index
cold_item_idx = np.load('./cold_item_index.npy', allow_pickle=True)
cold_item_idx.shape

(1410,)

In [8]:
print(f'{train_data[0]}\n{train_data[1]}\n{train_data[2]}')
print(train_data[0] + train_data[1] + train_data[2])

item_train_list = []
for item_t in train_data:
    for item in item_t[1:]:
        item_train_list.append(item)
print(f'item train: {len(np.unique(item_train_list))}')

item_test_list = []
for item_t in test_data:
    for item in item_t[1:]:
        item_test_list.append(item)
print(f'item test: {len(np.unique(item_test_list))}')

cold_start_item = [] 
unique_item_train = np.unique(item_train_list)
unique_item_test = np.unique(item_test_list)
for item in unique_item_test:
    if item not in unique_item_train:
        cold_start_item.append(item)
print(len(cold_start_item))

[18940, 3239, 6292, 669, 2269]
[152, 3210, 5195, 3075, 3966, 6806]
[2633, 1427, 4846, 4170, 3973]
[18940, 3239, 6292, 669, 2269, 152, 3210, 5195, 3075, 3966, 6806, 2633, 1427, 4846, 4170, 3973]
item train: 5640
item test: 705
705


In [9]:
# cl_centers_600 
cl_centers_600 = np.load('./cl_centers_600.npy', allow_pickle=True)
print(f'cl center shape: {cl_centers_600.shape}')
print(f'sample: {cl_centers_600[0]}')

# cl_cfmean_600
cl_cfmean_600 = np.load('./cl_cfmean_600.npy', allow_pickle=True)
print(f'shape cl_cfmean: {cl_cfmean_600.shape}')
print(f'sample: {cl_cfmean_600[0]}')

# clabel_600
clabel_600 = np.load('./clabel_600.npy', allow_pickle=True)
print(f'shape clabel_600: {clabel_600.shape}')
print(f'sample: {clabel_600[0]}')
print(f'num label cluster: {len(list(set(clabel_600)))}')

cl center shape: (7050, 5120)
sample: [ 0.          0.          0.         ... -0.12492786 -0.10371036
 -0.52446735]
shape cl_cfmean: (7050, 5120)
sample: [ 0.          0.          0.         ... -0.12492786 -0.10371036
 -0.52446735]
shape clabel_600: (7050,)
sample: 380
num label cluster: 600


In [10]:
from tqdm import tqdm 

# train.txt
def get_ui_list(train_data, task='train'):
    print(train_data)
    user_list_train = [a[0] for a in train_data]
    n_user = len(user_list_train)
    print(f'num user in {task}: {n_user}')
    item_list_train = []
    for sample in train_data:
        for item in sample[1:]:
            item_list_train.append(item)

    item_uni_list_train = list(set(item_list_train))
    n_item = len(item_uni_list_train)
    print(f'num item in {task}: {n_item}')
    return user_list_train, item_list_train, item_uni_list_train, n_user, n_item

user_list_train, item_list_train, item_uni_list_train, n_user, n_item = get_ui_list(train_data, task='train')
print('-'*50)
user_list_test, item_list_test, item_uni_list_test, n_user_test, n_item_test = get_ui_list(test_data, task='test')
print('-'*50)
user_list_val, item_list_val, item_uni_list_val, n_user_val, n_item_val = get_ui_list(val_data, task='val')


# sample user train list 
print('-'*50)
max_user_idx, min_user_idx = np.max(user_list_train), np.min(user_list_train)
print(f'max users idx: {max_user_idx}\nmin users idx: {min_user_idx}')

max_user_idx_test, min_user_idx_test = np.max(user_list_test), np.min(user_list_test)
print(f'max users idx test: {max_user_idx_test}\nmin users idx test: {min_user_idx_test}')

# sample item train list 
max_item_idx, min_item_idx = np.max(item_list_train), np.min(item_list_train)
print(f'max item idx: {max_item_idx}\nmin item idx: {min_item_idx}') 

# filter cold item
cold_list = [] 
print(f'n loop: {max_item_idx}')
for idx in tqdm(range(max_item_idx), desc='cold'):
    if idx not in item_uni_list_train:
        cold_list.append(idx)
print(f'num cold item: {len(cold_list)} ')

# cold_item_index 
cold_item_index = np.load('./cold_item_index.npy', allow_pickle=True)
print(f'shape cold item index: {cold_item_index.shape}')
print(f'full sample: {cold_item_index}')
print(f'cold start filter by hand: {cold_list}')
# check overlap 
n_item_overlap = (cold_item_index==np.array(cold_list)).sum()
print(
    'num overlap item between 2 cold index list: ',
    n_item_overlap
)
if n_item_overlap == cold_item_index.shape[0]:
    print('--------- 2 list overlap ---------')

# cold_missing_item_index
cold_missing_item_index = np.load('./cold_missing_item_index.npy', allow_pickle=True)
print(f'shape of cold missing item idx: {cold_missing_item_index.shape}')
print(f'partial sample: {cold_missing_item_index[:10]}')
print(f'percent cold missing item: {len(cold_missing_item_index) / n_item}')

# check location of cold missing item index
n_item_cold_train = 0
n_item_cold_test = 0 
n_item_cold_val = 0 
for idx in cold_missing_item_index:
    n_item_cold_train += (idx in item_list_train)
    n_item_cold_test += (idx in item_list_test)
    n_item_cold_val += (idx in item_list_val)
print(f'n item cold missing in train: {n_item_cold_train}')
print(f'n item cold missing in test: {n_item_cold_test}')
print(f'n item cold missing in val: {n_item_cold_val}')
print(f'percent n item cold missing in test/ n item in test: {n_item_cold_test/n_item_test}')
print(f'percent n item cold missing in val/ n item in val: {n_item_cold_val/n_item_val}')

# warm_missing_item_index 
print('-'*30, 'warm missing item index', '-'*30)
warm_missing_item_index = np.load('./warm_missing_item_index.npy', allow_pickle=True)
print(f'n warm missing item: {len(warm_missing_item_index)}')
print(f'partial sample: {warm_missing_item_index[:5]}')
print(f'percent warm missing item: {len(warm_missing_item_index) / n_item}')

# check location of warm missing item index 
n_item_warm_train = 0
n_item_warm_test = 0 
n_item_warm_val = 0 
for idx in warm_missing_item_index:
    n_item_warm_train += (idx in item_list_train)
    n_item_warm_test += (idx in item_list_test)
    n_item_warm_val += (idx in item_list_val)
print(f'n item warm missing in train: {n_item_warm_train}')
print(f'n item warm missing in test: {n_item_warm_test}')
print(f'n item warm missing in val: {n_item_warm_val}')
print(f'percent n item warm missing in train / n item in train: {n_item_warm_train / n_item}')

[[18940, 3239, 6292, 669, 2269], [152, 3210, 5195, 3075, 3966, 6806], [2633, 1427, 4846, 4170, 3973], [5505, 5834, 3742, 1996], [3525, 3229, 6499, 5953], [4404, 4699, 4670], [8559, 1587, 6915, 1203, 6645, 5803, 6423, 4771, 965], [14245, 6984, 6584], [17356, 6652, 4226, 3595, 4308, 3691, 5622], [12906, 4178, 4761, 1823, 3260, 5841, 5563, 6098, 1832, 1766, 1049], [238, 6368, 3850, 1913, 1440], [16205, 3221, 1403, 4845, 736, 2431], [13894, 2634, 756, 1839, 6721, 4930, 3716, 2741], [16352, 494, 3813, 2062], [5728, 3163, 2160, 2191, 3826, 5204, 2761], [9772, 4635, 5936, 6709, 5364, 426, 2445, 5444, 2352, 6860, 2439, 2054], [214, 83, 4422, 5612, 5353, 5027], [16239, 4178, 6934, 546, 999, 3385], [2315, 4178, 4839, 1348, 165, 3522, 3979, 6762, 4611, 6936, 915, 2632, 3920], [11921, 4178, 6890, 5520, 1015, 2151, 5059, 91, 4699, 6605, 4758, 3425, 2239, 4276, 792, 6697, 6585], [7494, 4178, 908, 871, 765, 4659, 6984, 6843, 2980, 3356, 2940], [9691, 4178, 4313, 3841, 68, 3505, 172, 3361, 627, 85, 53

cold: 100%|██████████| 7049/7049 [00:00<00:00, 20561.61it/s]


num cold item: 1410 
shape cold item index: (1410,)
full sample: [   1    6    8 ... 7045 7046 7048]
cold start filter by hand: [1, 6, 8, 9, 12, 14, 15, 20, 29, 31, 32, 33, 34, 38, 39, 42, 44, 48, 49, 51, 66, 70, 72, 90, 98, 119, 124, 130, 134, 142, 143, 144, 147, 148, 154, 158, 170, 187, 188, 194, 195, 202, 206, 214, 217, 220, 224, 227, 233, 244, 247, 252, 257, 260, 263, 269, 272, 274, 283, 286, 287, 291, 294, 295, 296, 302, 304, 311, 313, 317, 318, 346, 347, 371, 378, 379, 380, 388, 394, 396, 401, 405, 413, 414, 420, 429, 443, 444, 451, 456, 462, 464, 467, 470, 472, 475, 479, 487, 489, 495, 499, 502, 509, 517, 519, 521, 526, 527, 528, 530, 532, 533, 535, 539, 541, 543, 548, 555, 557, 558, 559, 572, 574, 578, 593, 597, 613, 620, 629, 634, 635, 641, 651, 660, 661, 683, 684, 693, 695, 718, 719, 722, 728, 729, 731, 733, 735, 737, 745, 751, 768, 771, 773, 776, 779, 783, 800, 811, 812, 818, 831, 834, 840, 841, 842, 843, 861, 862, 863, 864, 874, 875, 878, 880, 882, 888, 891, 910, 914, 916, 

In [11]:
# missing_image_index

# missing_image_index = np.load('./missing_image_index.npy', allow_pickle=True)
missing_image_index = np.load('./missing_text_index.npy', allow_pickle=True)

k = missing_image_index.sum(axis=1) == 0.
print(f'sample array size: {missing_image_index.shape[0]}') # train + test + val 
print(f'num item missing image: {k.sum()}')
print(f'sample:\n {missing_image_index[1:3, :]}')

# filter item in train missing image 
# v = missing_image_index.sum(axis=1).nonzero()
# v = list(v[0]) # convert to list 
# print(f'size v: {len(v)}')

# for idx in range(n_item):
#     if idx in item_list_train and idx not in v:
#          idx_miss_img_train.append(idx)
#     if idx in item_list_test and idx not in v:
#          idx_miss_img_test.append(idx)
#     if idx in item_list_val and idx not in v :
#          idx_miss_img_val.append(idx)
# print(len(idx_miss_img_train))
# print(len(idx_miss_img_test))
# print(len(idx_miss_img_val))

cnt = 0
idx_miss_img_train = []
idx_miss_img_test = []
idx_miss_img_val = []

out_idx = []
for idx, img_f in enumerate(missing_image_index):
    s = img_f.sum(axis=0)
    if s == 0.:
        cnt += 1
        cnt_t = 0
        if idx in item_uni_list_train:
            cnt_t += 1 
            idx_miss_img_train.append(idx)
        if idx in item_uni_list_test:
            cnt_t += 1 
            idx_miss_img_test.append(idx)
        if idx in item_uni_list_val:
            cnt_t += 1 
            idx_miss_img_val.append(idx)
        if cnt_t > 1:
            out_idx.append(idx)
    
# print(len(out_idx))
print(f'size miss img train: {len(idx_miss_img_train)}')
print(f'percent train: {len(idx_miss_img_train)/n_item}')
# print(len(idx_miss_img_test))
print(f'size miss img test: {len(idx_miss_img_test)}')
print(f'percent test: {len(idx_miss_img_test)/n_item_test}')
# print(len(idx_miss_img_val))
print(f'size misi img val: {len(idx_miss_img_val)}')
print(f'percent val: {len(idx_miss_img_val)/n_item_val}')
print(len(idx_miss_img_train) + len(idx_miss_img_test) + len(idx_miss_img_val)) # check lai 

sample array size: 7050
num item missing image: 2468
sample:
 [[ 1.2828709  -0.13528168 -1.0202562  ...  0.3137732  -0.1637356
   1.8551841 ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
size miss img train: 1991
percent train: 0.35301418439716314
size miss img test: 242
percent test: 0.3432624113475177
size misi img val: 235
percent val: 0.3333333333333333
2468


In [12]:
# missing_image_feat
missing_text_index = np.load('./missing_text_index.npy', allow_pickle=True)
# print(missing_text_index)
missing_text_feat = np.load('./missing_text_feat.npy', allow_pickle=True)
# print(f'num text miss: {(missing_text_feat.sum(axis=1) == 0.).sum()}')

missing_image_feat = np.load('./missing_image_feat.npy', allow_pickle=True)
# print(f'num image miss: {(missing_image_feat.sum(axis=1)==0.).sum()}')

missing_image_index = np.load('./missing_image_index.npy', allow_pickle=True)
print(missing_image_index)
print('-'*50)
print(missing_image_feat)
# test 
print((missing_image_index.sum(axis=1)==0.).sum())
print((missing_image_feat.sum(axis=1)==0.).sum())


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.72180003 0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
--------------------------------------------------
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.72180003 0.        ]
 ...
 [0.15380374 0.4127376  0.81748168 ... 0.30830754 0.64146708 0.55711264]
 [0.         0.         0.         ... 5.57450008 2.77469993 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
2467
2467
